In [1]:
pass

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1605607978346_0005,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# %load_ext autoreload

# %autoreload 2

# %matplotlib inline

In [3]:
import datetime

import numpy as np
import pandas as pd

import dask.dataframe as dd

# import matplotlib.pyplot as plt

# from tqdm import tqdm
# tqdm.pandas() # use progress_apply instead of apply

from dask.diagnostics import ProgressBar
ProgressBar().register()


In [4]:
import logging

import os
import sys
import json
# from pyspark.sql import SparkSession

from datetime import datetime

import boto3

from s3fs import S3FileSystem
fs = S3FileSystem()

from pyspark.sql import functions as F
from pyspark.sql.functions import explode, col

In [7]:
path = 's3://zeta-dcp-prod-private-tables/datacloud_cookie_emails_links_export/'

df = spark.read.option("inferSchema", True).orc(path + 'dt=2020-11-16/' + '/*')


In [8]:
df.printSchema()

root
 |-- cookie: string (nullable = true)
 |-- email_md5: string (nullable = true)
 |-- last_connected: timestamp (nullable = true)
 |-- last_updated: timestamp (nullable = true)
 |-- pos: integer (nullable = true)

In [9]:
df.show(n=5)

+--------------------+--------------------+-------------------+-------------------+---+
|              cookie|           email_md5|     last_connected|       last_updated|pos|
+--------------------+--------------------+-------------------+-------------------+---+
|33across::/yUFtV9...|cfe2364a978661fb3...|2020-09-16 20:00:01|2020-09-20 00:00:00|  1|
|33across::/yUFtV9...|1860b46bf57163647...|2020-09-30 14:30:11|2020-10-11 00:00:00|  1|
|33across::/yUFtV9...|2fbf6f0cca4fed127...|2020-09-30 14:30:11|2020-10-21 00:00:00|  2|
|33across::/yUFtV9...|67063e42e8d8ceff0...|2020-09-29 22:38:55|2020-10-02 00:00:00|  3|
|33across::/yUFtV9...|56c38f77452fa568c...|2020-08-17 02:15:32|2020-10-11 00:00:00|  4|
+--------------------+--------------------+-------------------+-------------------+---+
only showing top 5 rows

In [11]:
from pyspark.sql.functions import split

In [12]:
split_col = split(df['cookie'], '::')
df = df.withColumn('source', split_col.getItem(0))
df = df.withColumn('cookie_source', split_col.getItem(1))


In [13]:
df.printSchema()

root
 |-- cookie: string (nullable = true)
 |-- email_md5: string (nullable = true)
 |-- last_connected: timestamp (nullable = true)
 |-- last_updated: timestamp (nullable = true)
 |-- pos: integer (nullable = true)
 |-- source: string (nullable = true)
 |-- cookie_source: string (nullable = true)

In [14]:
df = df.select(df['source'], df['cookie_source'], df['email_md5'], df['last_connected'], df['last_updated'], df['pos'])



In [15]:
df.printSchema()

root
 |-- source: string (nullable = true)
 |-- cookie_source: string (nullable = true)
 |-- email_md5: string (nullable = true)
 |-- last_connected: timestamp (nullable = true)
 |-- last_updated: timestamp (nullable = true)
 |-- pos: integer (nullable = true)

In [ ]:
df.show()

In [16]:
path = 's3://zeta-dc-ml/ai-audiences/datacloud_cookie_emails_links_export/cookie_email_staging/'

df.write.parquet(path + 'dt=2020-11-16/')

In [22]:
path = 's3://zeta-dc-ml/ai-audiences/datacloud_cookie_emails_links_export/cookie_email_staging/'

df = spark.read.option("inferSchema", True).parquet(path +'dt=2020-11-16/' + '*.parquet')


In [23]:
df.printSchema()

root
 |-- source: string (nullable = true)
 |-- cookie_source: string (nullable = true)
 |-- email_md5: string (nullable = true)
 |-- last_connected: timestamp (nullable = true)
 |-- last_updated: timestamp (nullable = true)
 |-- pos: integer (nullable = true)

In [24]:
df.show(n=2)

+--------+--------------------+--------------------+-------------------+-------------------+---+
|  source|       cookie_source|           email_md5|     last_connected|       last_updated|pos|
+--------+--------------------+--------------------+-------------------+-------------------+---+
|33across|/yUFtV9NXan/i2F4A...|cfe2364a978661fb3...|2020-09-16 20:00:01|2020-09-20 00:00:00|  1|
|33across|/yUFtV9NYcD/i2F4A...|1860b46bf57163647...|2020-09-30 14:30:11|2020-10-11 00:00:00|  1|
+--------+--------------------+--------------------+-------------------+-------------------+---+
only showing top 2 rows

In [25]:
df_temp = (df
 .dropDuplicates(['cookie_source'])
 .groupBy("source")
 .count()
 .orderBy(col("count").desc())
)

In [27]:
pdf = df_temp.toPandas()

In [28]:
pdf.head()

     source       count
0    disqus  2045955820
1      zync  1472134625
2    sizmek   759915579
3  appnexus   682379988
4  33across   667707987

In [29]:
df_temp_2 = (df
 .dropDuplicates(['cookie_source', 'email_md5'])
 .groupBy("source")
 .count()
 .orderBy(col("count").desc())
)

In [30]:
pdf2 = df_temp_2.toPandas()